In [ ]:
%pip install git+https://github.com/vanderschaarlab/temporai.git
import pandas as pd
import numpy as np
from random import randrange, uniform

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
colunas = ['eventdeath', 'survival', 'timerecurrence', 'chemo', 'hormonal', 'amputation', 'histtype', 'diam', 'posnodes', 'grade', 'angioinv', 'lymphinfil' ]
df = pd.read_csv('/content/drive/MyDrive/Data/nki_cleaned.csv', usecols=colunas)

index_count = []
random_num = []

for i in range(1, len(df.index)+1):
  index_count.append(i)
  random_num.append(randrange(1, 20))

df['sample_idx'] = index_count
df['time_idx'] = random_num

In [ ]:
time_series_df = df.copy()
time_series_df.set_index(keys=["sample_idx", "time_idx"], drop=True, inplace=True)

In [ ]:
time_series_df

In [ ]:
event_df = df.copy()
event_df.set_index(keys=["sample_idx"], drop=True, inplace=True)

event_df['event'] = event_df[['time_idx', 'eventdeath']].apply(tuple, axis=1)
event_df.drop(event_df.iloc[:, :13], inplace=True, axis=1)

In [ ]:
event_df

In [ ]:
from tempor.data.dataset import TimeToEventAnalysisDataset

data = TimeToEventAnalysisDataset(
    time_series=time_series_df,
    targets=event_df,
)

In [ ]:
from tempor import plugin_loader

model = plugin_loader.get("time_to_event.dynamic_deephit", n_iter=50)
model.fit(data)
model.predict(data, horizons=[0.25, 0.50, 0.75])